<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import pandas as pd
df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

In [2]:
print(df.shape)
print(df.head())
print(df.columns)

(7043, 21)
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Co

In [3]:
print([df[x].value_counts() for x in df.columns])

[2988-QRAJY    1
0906-QVPMS    1
2141-RRYGO    1
6140-QNRQQ    1
8058-JMEQO    1
             ..
3462-BJQQA    1
8752-IMQOS    1
9928-BZVLZ    1
7576-OYWBN    1
5414-OFQCB    1
Name: customerID, Length: 7043, dtype: int64, Male      3555
Female    3488
Name: gender, dtype: int64, 0    5901
1    1142
Name: SeniorCitizen, dtype: int64, No     3641
Yes    3402
Name: Partner, dtype: int64, No     4933
Yes    2110
Name: Dependents, dtype: int64, 1     613
72    362
2     238
3     200
4     176
     ... 
28     57
39     56
44     51
36     50
0      11
Name: tenure, Length: 73, dtype: int64, Yes    6361
No      682
Name: PhoneService, dtype: int64, No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64, Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64, No                     3498
Yes                    2019
No internet service    1526
Name: OnlineSecurity, dtype: int64, No                  

In [4]:
import numpy
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [5]:
numpy.random.seed(42)
X = df.drop(columns=['Churn', 'customerID']).values
y = df['Churn'].values


onehot = OneHotEncoder(handle_unknown='ignore')

X = onehot.fit_transform(X)
X = X.toarray()

inputs = X.shape[1]

def make_model():
        model = Sequential()
        model.add(Dense(32, input_shape=(inputs,), activation='relu', ))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(1))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

model = KerasClassifier(build_fn=make_model, verbose=1)

parameters = {'batch_size': [5, 10, 20],
             #'optimizer': ['rmsprop', 'adam'],
             #'learning_rate': [0.01, 0.03, 0.05],
             #'validation_split': [0.20, 0.25, 0.33]
             }

grid = GridSearchCV(estimator=model, param_grid=parameters, n_jobs = -1)
grid_result = grid.fit(X, y, epochs=30)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Train on 7043 samples
Epoch 1/30
7043/7043 [==============================] - 1s 181us/sample - loss: 0.5213 - accuracy: 0.7781
Epoch 2/30
7043/7043 [==============================] - 1s 134us/sample - loss: 0.4110 - accuracy: 0.8352
Epoch 3/30
7043/7043 [==============================] - 1s 135us/sample - loss: 0.3950 - accuracy: 0.8536
Epoch 4/30
7043/7043 [==============================] - 1s 135us/sample - loss: 0.2551 - accuracy: 0.9188
Epoch 5/30
7043/7043 [==============================] - 1s 134us/sample - loss: 0.1500 - accuracy: 0.9622
Epoch 6/30
7043/7043 [==============================] - 1s 133us/sample - loss: 0.1573 - accuracy: 0.9683
Epoch 7/30
7043/7043 [==============================] - 1s 129us/sample - loss: 0.0912 - accuracy: 0.9766
Epoch 8/30
7043/7043 [==============================] - 1s 129us/sample - loss: 0.0730 - accuracy: 0.9820
Epoch 9/30
7043/7043 [==============================] - 1s 136us/sample - loss: 0.0660 - accuracy: 0.9840
Epoch 10/30
7043/7043 [=

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?